In [55]:
!pip install -r requirements.txt

# Requirements -- EJECUTAR ESTA Celda ^^^^^^^^^^
# Necesario para ejecutar las celdas ^^^^^^^^^^^

# Optimización del Cultivo en una Granja

## Contexto

#### En una granja de Segovia se cultivan diferentes tipos de vegetales durante la temporada de verano. Los cultivos deben planificarse teniendo en cuenta la disponibilidad de agua, fertilizantes y superficie cultivable, con el objetivo de maximizar el beneficio total.

##### Los cultivos disponibles son:

1. **Tomates**.
2. **Pimientos**.
3. **Lechugas**.
4. **Calabacines**.
5. **Berenjenas**.
6. **Zanahorias**.


---

## Datos del Problema

| Cultivo      | Beneficio por Hectárea (€) | Agua (m³/hectárea) | Fertilizantes (kg/hectárea) | Superficie mínima requerida (hectáreas) |
|--------------|----------------------------|---------------------|-----------------------------|------------------------------------------|
| Tomates      | 2000                       | 300                 | 50                          | 10                                       |
| Pimientos    | 2500                       | 400                 | 60                          | 15                                       |
| Lechugas     | 1500                       | 200                 | 40                          | 5                                        |
| Calabacines  | 1800                       | 350                 | 55                          | 8                                        |
| Berenjenas   | 2200                       | 450                 | 70                          | 10                                       |
| Zanahorias   | 1700                       | 300                 | 50                          | 7                                        |

### Recursos Disponibles

- **Superficie total disponible**: 100 hectáreas.
- **Agua disponible**: 30,000 m³.
- **Fertilizantes disponibles**: 5,000 kg.

---

## Formulación del Problema
## Definición de las Variables

Las variables de decisión representan la superficie dedicada a cada tipo de cultivo en hectáreas. Estas se definen de la siguiente manera:

- $X_1$: Hectáreas dedicadas al cultivo de **Tomates**.
- $X_2$: Hectáreas dedicadas al cultivo de **Pimientos**.
- $X_3$: Hectáreas dedicadas al cultivo de **Lechugas**.
- $X_4$: Hectáreas dedicadas al cultivo de **Calabacines**.
- $X_5$: Hectáreas dedicadas al cultivo de **Berenjenas**.
- $X_6$: Hectáreas dedicadas al cultivo de **Zanahorias**.

### Restricciones Adicionales:
- $X_i$ debe ser mayor o igual al mínimo de superficie requerido para cada cultivo:
  - $X_1 \geq 10$
  - $X_2 \geq 15$
  - $X_3 \geq 5$
  - $X_4 \geq 8$
  - $X_5 \geq 10$
  - $X_6 \geq 7$
- Todas las variables $X_i$ son no negativas ($X_i \geq 0$).

Maximizar:  
$$ 
Z = 2000X_1 + 2500X_2 + 1500X_3 + 1800X_4 + 2200X_5 + 1700X_6 
$$

Sujeto a las siguientes restricciones:  
$$
\begin{aligned}
    X_1 + X_2 + X_3 + X_4 + X_5 + X_6 &\leq 100 \\
    300X_1 + 400X_2 + 200X_3 + 350X_4 + 450X_5 + 300X_6 &\leq 30,000 \\
    50X_1 + 60X_2 + 40X_3 + 55X_4 + 70X_5 + 50X_6 &\leq 5,000 \\
    X_1 &\geq 10 \\
    X_2 &\geq 15 \\
    X_3 &\geq 5 \\
    X_4 &\geq 8 \\
    X_5 &\geq 10 \\
    X_6 &\geq 7 \\
    X_1, X_2, X_3, X_4, X_5, X_6 &\geq 0
\end{aligned}
$$

---



### El objetivo del problema es determinar cuántas hectáreas se deben dedicar a cada tipo de cultivo para maximizar el beneficio total, respetando las limitaciones de recursos y las exigencias mínimas de superficie para cada tipo de vegetal. ###

In [56]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Variables de decisión (hectáreas dedicadas a cada cultivo)
model.x1 = Var(within=NonNegativeReals)  # Tomates
model.x2 = Var(within=NonNegativeReals)  # Pimientos
model.x3 = Var(within=NonNegativeReals)  # Lechugas
model.x4 = Var(within=NonNegativeReals)  # Calabacines
model.x5 = Var(within=NonNegativeReals)  # Berenjenas
model.x6 = Var(within=NonNegativeReals)  # Zanahorias

# Función objetivo: Maximizar beneficios
model.obj = Objective(
    expr=2000*model.x1 + 2500*model.x2 + 1500*model.x3 +
         1800*model.x4 + 2200*model.x5 + 1700*model.x6,
    sense=maximize
)

# Restricciones
model.constr1 = Constraint(expr=model.x1 + model.x2 + model.x3 +
                           model.x4 + model.x5 + model.x6 <= 100)
model.constr2 = Constraint(expr=300*model.x1 + 400*model.x2 + 200*model.x3 +
                           350*model.x4 + 450*model.x5 + 300*model.x6 <= 30000)
model.constr3 = Constraint(expr=50*model.x1 + 60*model.x2 + 40*model.x3 +
                           55*model.x4 + 70*model.x5 + 50*model.x6 <= 5000)
model.constr4 = Constraint(expr=model.x1 >= 10)
model.constr5 = Constraint(expr=model.x2 >= 15)
model.constr6 = Constraint(expr=model.x3 >= 5)
model.constr7 = Constraint(expr=model.x4 >= 8)
model.constr8 = Constraint(expr=model.x5 >= 10)
model.constr9 = Constraint(expr=model.x6 >= 7)

# Resolver el modelo
solver = SolverFactory('glpk')
result = solver.solve(model)

# Mostrar resultados
print(f"Estado de la solución: {result.solver.status}")
print(f"Estado de la optimización: {result.solver.termination_condition}\n")

print(f"Ganancia máxima: {model.obj():.2f} €\n")
print(f"Producción óptima:")
print(f"Tomates: {model.x1():.2f} hectáreas")
print(f"Pimientos: {model.x2():.2f} hectáreas")
print(f"Lechugas: {model.x3():.2f} hectáreas")
print(f"Calabacines: {model.x4():.2f} hectáreas")
print(f"Berenjenas: {model.x5():.2f} hectáreas")
print(f"Zanahorias: {model.x6():.2f} hectáreas")

Estado de la solución: ok
Estado de la optimización: optimal

Ganancia máxima: 187550.00 €

Producción óptima:
Tomates: 31.00 hectáreas
Pimientos: 15.00 hectáreas
Lechugas: 26.50 hectáreas
Calabacines: 8.00 hectáreas
Berenjenas: 10.00 hectáreas
Zanahorias: 7.00 hectáreas


# Problema Simplificado: Optimización con Dos Variables de Decisión

## Contexto

Este problema es una simplificación del modelo inicial, donde consideramos solo dos cultivos: **Tomates ($x_1$)** y **Pimientos ($x_2$)**. El objetivo sigue siendo **maximizar el beneficio total**, respetando las restricciones de recursos disponibles (superficie, agua, fertilizantes) y las superficies mínimas requeridas para cada cultivo.

---

## Formulación del Problema

### Variables de Decisión

- $x_1$: Hectáreas dedicadas al cultivo de **Tomates**.
- $x_2$: Hectáreas dedicadas al cultivo de **Pimientos**.

---

### Función Objetivo

Maximizar el beneficio total:
$$
Z = 2000x_1 + 2500x_2
$$

---

### Restricciones

1. **Capacidad Total**:
   $$
   x_1 + x_2 \leq 100
   $$

2. **Agua Disponible**:
   $$
   300x_1 + 400x_2 \leq 30,000
   $$

3. **Fertilizantes Disponibles**:
   $$
   50x_1 + 60x_2 \leq 5,000
   $$

4. **Superficie Mínima**:
   $$
   x_1 \geq 10, \quad x_2 \geq 15
   $$

5. **No Negatividad**:
   $$
   x_1 \geq 0, \quad x_2 \geq 0
   $$

---

## Resolución Gráfica

El problema será resuelto gráficamente:
- Se representarán las restricciones como líneas en un plano cartesiano.
- La **región factible** será el área donde se cumplan todas las restricciones.
- La **función objetivo** será representada con líneas de nivel para identificar el punto óptimo que maximiza el beneficio total.

---

## Objetivo

Determinar gráficamente las hectáreas óptimas que deben dedicarse al cultivo de **Tomates** y **Pimientos** para **maximizar el beneficio total**, respetando las restricciones de recursos y superficies mínimas.

In [57]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros del problema
beneficio_tomates = 2000
beneficio_pimientos = 2500
agua_tomates = 300
agua_pimientos = 400
fertilizantes_tomates = 50
fertilizantes_pimientos = 60
superficie_total = 100
agua_total = 30000
fertilizantes_total = 5000
superficie_min_tomates = 10
superficie_min_pimientos = 15

# Crear las restricciones
x1 = np.linspace(0, 100, 500)

# Restricciones
r1 = superficie_total - x1  # x1 + x2 <= 100
r2 = (agua_total - agua_tomates * x1) / agua_pimientos  # 300x1 + 400x2 <= 30000
r3 = (fertilizantes_total - fertilizantes_tomates * x1) / fertilizantes_pimientos  # 50x1 + 60x2 <= 5000

# Limites mínimos
r4 = np.full_like(x1, superficie_min_tomates)  # x1 >= 10
r5 = np.full_like(x1, superficie_min_pimientos)  # x2 >= 15

# Graficar la región factible
plt.figure(figsize=(8, 8))
plt.plot(x1, r1, label=r"$x_1 + x_2 \leq 100$")
plt.plot(x1, r2, label=r"$300x_1 + 400x_2 \leq 30000$")
plt.plot(x1, r3, label=r"$50x_1 + 60x_2 \leq 5000$")
plt.axhline(y=superficie_min_pimientos, color="purple", linestyle="--", label=r"$x_2 \geq 15$")
plt.axvline(x=superficie_min_tomates, color="orange", linestyle="--", label=r"$x_1 \geq 10$")

# Región factible (intersección)
plt.fill_between(x1, 0, np.minimum.reduce([r1, r2, r3]), color="gray", alpha=0.3, label="Región Factible")

# Poner límites en el gráfico
plt.xlim(0, 100)
plt.ylim(0, 100)

# Función objetivo
x1_obj = np.linspace(0, 100, 500)
for z in [50000, 100000, 150000]:
    x2_obj = (z - beneficio_tomates * x1_obj) / beneficio_pimientos
    plt.plot(x1_obj, x2_obj, linestyle="--", label=f"$Z = {z}$")

# Personalizar el gráfico
plt.xlabel(r"$x_1$ (Tomates en hectáreas)")
plt.ylabel(r"$x_2$ (Pimientos en hectáreas)")
plt.title("Optimización Gráfica con 2 Variables de Decisión")
plt.legend(loc="upper right")
plt.grid(True)

# Mostrar el gráfico
plt.show()

RecursionError: maximum recursion depth exceeded

<Figure size 800x800 with 1 Axes>

# Problema Dual: Optimización del Cultivo en una Granja

## Contexto del Problema Dual

En el problema dual, buscamos determinar el valor asociado a cada restricción del problema primal. Esto implica asignar un precio sombra a los recursos disponibles (agua, fertilizantes, superficie total) y al cumplimiento de las superficies mínimas requeridas por cada cultivo. El objetivo es minimizar el coste total de los recursos utilizados mientras se asegura que las condiciones de beneficio por hectárea se cumplen para cada cultivo.

---

## Formulación del Problema Dual

Minimizar:  
$$
W = 100Y_1 + 30,000Y_2 + 5,000Y_3 + 10Y_4 + 15Y_5 + 5Y_6 + 8Y_7 + 10Y_8 + 7Y_9
$$

Sujeto a las siguientes restricciones:  
$$
\begin{aligned}
    Y_1 + 300Y_2 + 50Y_3 + Y_4 &\geq 2000 & \text{(Tomates)} \\
    Y_1 + 400Y_2 + 60Y_3 + Y_5 &\geq 2500 & \text{(Pimientos)} \\
    Y_1 + 200Y_2 + 40Y_3 + Y_6 &\geq 1500 & \text{(Lechugas)} \\
    Y_1 + 350Y_2 + 55Y_3 + Y_7 &\geq 1800 & \text{(Calabacines)} \\
    Y_1 + 450Y_2 + 70Y_3 + Y_8 &\geq 2200 & \text{(Berenjenas)} \\
    Y_1 + 300Y_2 + 50Y_3 + Y_9 &\geq 1700 & \text{(Zanahorias)} \\
    Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9 &\geq 0
\end{aligned}
$$

---

## Explicación de las Variables

- $Y_1$: Precio sombra por hectárea de superficie cultivable.
- $Y_2$: Precio sombra por m³ de agua disponible.
- $Y_3$: Precio sombra por kg de fertilizantes disponibles.
- $Y_4$: Valor asociado al cumplimiento de la superficie mínima para tomates.
- $Y_5$: Valor asociado al cumplimiento de la superficie mínima para pimientos.
- $Y_6$: Valor asociado al cumplimiento de la superficie mínima para lechugas.
- $Y_7$: Valor asociado al cumplimiento de la superficie mínima para calabacines.
- $Y_8$: Valor asociado al cumplimiento de la superficie mínima para berenjenas.
- $Y_9$: Valor asociado al cumplimiento de la superficie mínima para zanahorias.

---

## Interpretación

El problema dual nos ayuda a determinar el valor económico de los recursos y restricciones del problema primal. Cada variable $Y_i$ representa cuánto aumentaría el coste total si se incrementara la disponibilidad de recursos o se modificaran las restricciones mínimas en el primal. Esto permite evaluar la sensibilidad del sistema y priorizar inversiones en recursos o ajustes en los cultivos.

In [58]:
from pyomo.environ import *

# Crear el modelo
dual_model = ConcreteModel()

# Variables duales
dual_model.y1 = Var(within=NonNegativeReals)  # Precio sombra para superficie cultivable
dual_model.y2 = Var(within=NonNegativeReals)  # Precio sombra para agua
dual_model.y3 = Var(within=NonNegativeReals)  # Precio sombra para fertilizantes
dual_model.y4 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de tomates
dual_model.y5 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de pimientos
dual_model.y6 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de lechugas
dual_model.y7 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de calabacines
dual_model.y8 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de berenjenas
dual_model.y9 = Var(within=NonNegativeReals)  # Precio sombra para superficie mínima de zanahorias

# Función objetivo: Minimizar el coste total de recursos y restricciones
dual_model.obj = Objective(
    expr=100*dual_model.y1 + 30000*dual_model.y2 + 5000*dual_model.y3 +
         10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
         8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9,
    sense=minimize
)

# Restricciones duales: Asegurar que el coste asignado a cada cultivo no sea menor que su beneficio
dual_model.constr1 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y4 >= 2000)  # Tomates
dual_model.constr2 = Constraint(expr=dual_model.y1 + 400*dual_model.y2 + 60*dual_model.y3 + dual_model.y5 >= 2500)  # Pimientos
dual_model.constr3 = Constraint(expr=dual_model.y1 + 200*dual_model.y2 + 40*dual_model.y3 + dual_model.y6 >= 1500)  # Lechugas
dual_model.constr4 = Constraint(expr=dual_model.y1 + 350*dual_model.y2 + 55*dual_model.y3 + dual_model.y7 >= 1800)  # Calabacines
dual_model.constr5 = Constraint(expr=dual_model.y1 + 450*dual_model.y2 + 70*dual_model.y3 + dual_model.y8 >= 2200)  # Berenjenas
dual_model.constr6 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y9 >= 1700)  # Zanahorias

# Resolver el modelo
solver = SolverFactory('glpk')
result = solver.solve(dual_model)

# Mostrar resultados
print(f"Estado de la solución: {result.solver.status}")
print(f"Estado de la optimización: {result.solver.termination_condition}\n")

print(f"Coste total mínimo: {dual_model.obj():.2f} €\n")
print(f"Precios sombra (valores duales):")
print(f"Superficie cultivable (Y1): {dual_model.y1():.2f} €/hectárea")
print(f"Agua (Y2): {dual_model.y2():.2f} €/m³")
print(f"Fertilizantes (Y3): {dual_model.y3():.2f} €/kg")
print(f"Superficie mínima de tomates (Y4): {dual_model.y4():.2f} €")
print(f"Superficie mínima de pimientos (Y5): {dual_model.y5():.2f} €")
print(f"Superficie mínima de lechugas (Y6): {dual_model.y6():.2f} €")
print(f"Superficie mínima de calabacines (Y7): {dual_model.y7():.2f} €")
print(f"Superficie mínima de berenjenas (Y8): {dual_model.y8():.2f} €")
print(f"Superficie mínima de zanahorias (Y9): {dual_model.y9():.2f} €")

Estado de la solución: ok
Estado de la optimización: optimal

Coste total mínimo: 113300.00 €

Precios sombra (valores duales):
Superficie cultivable (Y1): 0.00 €/hectárea
Agua (Y2): 0.00 €/m³
Fertilizantes (Y3): 0.00 €/kg
Superficie mínima de tomates (Y4): 2000.00 €
Superficie mínima de pimientos (Y5): 2500.00 €
Superficie mínima de lechugas (Y6): 1500.00 €
Superficie mínima de calabacines (Y7): 1800.00 €
Superficie mínima de berenjenas (Y8): 2200.00 €
Superficie mínima de zanahorias (Y9): 1700.00 €


## Resultados del Problema Dual

### Coste Total Mínimo

El coste total mínimo obtenido es de **113,300 €**, que corresponde al valor óptimo del problema dual. Este valor indica el coste total asociado a satisfacer las demandas mínimas de superficie cultivable, agua, fertilizantes y las superficies mínimas requeridas para cada cultivo.

### Precios Sombra

Los precios sombra ($Y_i$) :



| Recurso / Restricción               | Precio Sombra (€) |
|-------------------------------------|-------------------|
| Superficie cultivable ($Y_1$)   | 0.00              |
| Agua ($Y_2$)                    | 0.00              |
| Fertilizantes ($Y_3$)           | 0.00              |
| Superficie mínima de tomates ($Y_4$) | 2,000.00        |
| Superficie mínima de pimientos ($Y_5$) | 2,500.00        |
| Superficie mínima de lechugas ($Y_6$)  | 1,500.00        |
| Superficie mínima de calabacines ($Y_7$) | 1,800.00        |
| Superficie mínima de berenjenas ($Y_8$) | 2,200.00        |
| Superficie mínima de zanahorias ($Y_9$) | 1,700.00        |

---

## Interpretación de los Resultados

### Recursos Sin Escasez

Los precios sombra asociados a los recursos básicos (superficie cultivable, agua y fertilizantes) son **0**. Esto significa que no se alcanzó el límite total de estos recursos en la solución óptima del primal. En otras palabras, aumentar la cantidad de estos recursos no tendría un impacto significativo en el beneficio total, ya que no son cuellos de botella.

### Restricciones Activas

Los precios sombra positivos ($Y_4$ a $Y_9$) están asociados a las superficies mínimas requeridas para cada cultivo. Esto indica que estas restricciones son activas en la solución óptima y, por lo tanto, están limitando el beneficio total. 

#### Interpretación de los Precios Sombra Positivos

- **Superficie mínima de tomates ($Y_4$):** Cada hectárea adicional para tomates contribuiría con 2,000 € al coste total del problema dual.
- **Superficie mínima de pimientos ($Y_5$):** Este cultivo tiene el mayor precio sombra, con 2,500 €. Incrementar la superficie dedicada a pimientos tendría un impacto considerable en la solución óptima.
- **Lechugas, calabacines, berenjenas y zanahorias ($Y_6$ a $Y_9$):** Cada una de estas restricciones tiene un precio sombra positivo, lo que implica que cumplir con sus superficies mínimas requeridas es esencial para mantener la solución óptima.

### Relación con el Primal

El valor óptimo del problema dual (113,300 €) es consistente con el beneficio total obtenido en el problema primal (187,550 €), según el **Teorema de la Dualidad Fuerte**, que garantiza que ambos valores deben coincidir si la solución es óptima.

---

## Conclusiones

1. **Recursos disponibles:** Los recursos totales (superficie cultivable, agua y fertilizantes) no son limitantes en la solución óptima, lo que sugiere que hay capacidad adicional para producir más cultivos si fuera necesario.

2. **Restricciones activas:** Las restricciones de superficies mínimas requeridas para cada cultivo son críticas en el problema primal. Esto se refleja en los precios sombra positivos, que representan el valor económico de cumplir con estas restricciones.

3. **Optimización del sistema:** Este análisis sugiere que, para incrementar el beneficio total, sería más valioso relajar las restricciones de superficie mínima en cultivos como pimientos y berenjenas, ya que tienen los precios sombra más altos.

El análisis del problema dual proporciona una visión complementaria del problema primal, permitiendo identificar oportunidades para mejorar la planificación de recursos y cultivos en temporadas futuras.

# Análisis de Sensibilidad del Problema Dual

## Contexto

El análisis de sensibilidad permite evaluar cómo los cambios en los coeficientes de la función objetivo o en las restricciones afectan el coste total mínimo y los precios sombra. Esto es útil para identificar los recursos más críticos y entender la robustez del modelo dual.

En este análisis, se han considerado tres recursos clave:
1. **Superficie cultivable** ($Y_1$).
2. **Agua disponible** ($Y_2$).
3. **Fertilizantes disponibles** ($Y_3$).

---

## Metodología

Se evalúa el impacto de modificar los coeficientes de estos recursos en la función objetivo:
$$
W = Y_1 \cdot C_{\text{Superficie}} + Y_2 \cdot C_{\text{Agua}} + Y_3 \cdot C_{\text{Fertilizantes}} + \sum_{i=4}^{9} Y_i \cdot C_{\text{Superficies Mínimas}}
$$

Donde:
- $C_{\text{Superficie}}, C_{\text{Agua}}, C_{\text{Fertilizantes}}$ son los coeficientes asociados a cada recurso.
- $Y_i$ representa el precio sombra del recurso $i$.

### Cambios Analizados

1. **Superficie cultivable**:
   - Se evalúan los valores: $50$, $100$, $150$ €/ha.

2. **Agua disponible**:
   - Se evalúan los valores: $25,000$, $30,000$, $35,000$ €/m³.

3. **Fertilizantes disponibles**:
   - Se evalúan los valores: $4,000$, $5,000$, $6,000$ €/kg.

---

In [59]:
from pyomo.environ import *

# Crear el modelo dual base
def crear_modelo_dual():
    dual_model = ConcreteModel()
    
    # Variables duales
    dual_model.y1 = Var(within=NonNegativeReals)  # Superficie cultivable
    dual_model.y2 = Var(within=NonNegativeReals)  # Agua
    dual_model.y3 = Var(within=NonNegativeReals)  # Fertilizantes
    dual_model.y4 = Var(within=NonNegativeReals)  # Superficie mínima de tomates
    dual_model.y5 = Var(within=NonNegativeReals)  # Superficie mínima de pimientos
    dual_model.y6 = Var(within=NonNegativeReals)  # Superficie mínima de lechugas
    dual_model.y7 = Var(within=NonNegativeReals)  # Superficie mínima de calabacines
    dual_model.y8 = Var(within=NonNegativeReals)  # Superficie mínima de berenjenas
    dual_model.y9 = Var(within=NonNegativeReals)  # Superficie mínima de zanahorias

    # Función objetivo: Minimizar costes
    dual_model.obj = Objective(
        expr=100*dual_model.y1 + 30000*dual_model.y2 + 5000*dual_model.y3 +
             10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
             8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9,
        sense=minimize
    )

    # Restricciones duales
    dual_model.constr1 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y4 >= 2000)  # Tomates
    dual_model.constr2 = Constraint(expr=dual_model.y1 + 400*dual_model.y2 + 60*dual_model.y3 + dual_model.y5 >= 2500)  # Pimientos
    dual_model.constr3 = Constraint(expr=dual_model.y1 + 200*dual_model.y2 + 40*dual_model.y3 + dual_model.y6 >= 1500)  # Lechugas
    dual_model.constr4 = Constraint(expr=dual_model.y1 + 350*dual_model.y2 + 55*dual_model.y3 + dual_model.y7 >= 1800)  # Calabacines
    dual_model.constr5 = Constraint(expr=dual_model.y1 + 450*dual_model.y2 + 70*dual_model.y3 + dual_model.y8 >= 2200)  # Berenjenas
    dual_model.constr6 = Constraint(expr=dual_model.y1 + 300*dual_model.y2 + 50*dual_model.y3 + dual_model.y9 >= 1700)  # Zanahorias

    return dual_model

# Realizar el análisis de sensibilidad
def analisis_sensibilidad():
    # Cambios en los coeficientes de la función objetivo
    cambios_objetivo = {
        'Superficie cultivable': [50, 100, 150],
        'Agua': [25000, 30000, 35000],
        'Fertilizantes': [4000, 5000, 6000]
    }

    resultados = []

    for recurso, valores in cambios_objetivo.items():
        for valor in valores:
            dual_model = crear_modelo_dual()

            # Ajustar el coeficiente correspondiente en la función objetivo
            if recurso == 'Superficie cultivable':
                dual_model.obj.expr = (
                    valor*dual_model.y1 + 30000*dual_model.y2 + 5000*dual_model.y3 +
                    10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
                    8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9
                )
            elif recurso == 'Agua':
                dual_model.obj.expr = (
                    100*dual_model.y1 + valor*dual_model.y2 + 5000*dual_model.y3 +
                    10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
                    8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9
                )
            elif recurso == 'Fertilizantes':
                dual_model.obj.expr = (
                    100*dual_model.y1 + 30000*dual_model.y2 + valor*dual_model.y3 +
                    10*dual_model.y4 + 15*dual_model.y5 + 5*dual_model.y6 +
                    8*dual_model.y7 + 10*dual_model.y8 + 7*dual_model.y9
                )

            # Resolver el modelo ajustado
            solver = SolverFactory('glpk')
            result = solver.solve(dual_model, tee=False)

            # Guardar resultados
            resultados.append({
                'Recurso': recurso,
                'Valor': valor,
                'Coste Total Mínimo (€)': dual_model.obj(),
                'Precio Sombra Superficie (€)': dual_model.y1(),
                'Precio Sombra Agua (€)': dual_model.y2(),
                'Precio Sombra Fertilizantes (€)': dual_model.y3()
            })

    return resultados

# Ejecutar el análisis de sensibilidad
sensibilidad = analisis_sensibilidad()

# Mostrar resultados
print("\n🔹 Análisis de Sensibilidad 🔹")
for resultado in sensibilidad:
    print(f"Recurso: {resultado['Recurso']}, Valor: {resultado['Valor']}, "
          f"Coste Total Mínimo: {resultado['Coste Total Mínimo (€)']:.2f}, "
          f"Precio Sombra Superficie: {resultado['Precio Sombra Superficie (€)']:.2f}, "
          f"Precio Sombra Agua: {resultado['Precio Sombra Agua (€)']:.2f}, "
          f"Precio Sombra Fertilizantes: {resultado['Precio Sombra Fertilizantes (€)']:.2f}")


🔹 Análisis de Sensibilidad 🔹
Recurso: Superficie cultivable, Valor: 50, Coste Total Mínimo: 105800.00, Precio Sombra Superficie: 1500.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Superficie cultivable, Valor: 100, Coste Total Mínimo: 113300.00, Precio Sombra Superficie: 0.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Superficie cultivable, Valor: 150, Coste Total Mínimo: 113300.00, Precio Sombra Superficie: 0.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Agua, Valor: 25000, Coste Total Mínimo: 113300.00, Precio Sombra Superficie: 0.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Agua, Valor: 30000, Coste Total Mínimo: 113300.00, Precio Sombra Superficie: 0.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Agua, Valor: 35000, Coste Total Mínimo: 113300.00, Precio Sombra Superficie: 0.00, Precio Sombra Agua: 0.00, Precio Sombra Fertilizantes: 0.00
Recurso: Fe

## Resultados

### Tabla de Resultados del Análisis de Sensibilidad

| Recurso                  | Valor       | Coste Total Mínimo (€) | $Y_1$ (€/ha) | $Y_2$ (€/m³) | $Y_3$ (€/kg) |
|--------------------------|-------------|-------------------------|--------------|--------------|--------------|
| Superficie cultivable    | 50          | 105,800.00             | 1500.00      | 0.00         | 0.00         |
|                          | 100         | 113,300.00             | 0.00         | 0.00         | 0.00         |
|                          | 150         | 113,300.00             | 0.00         | 0.00         | 0.00         |
| Agua disponible          | 25,000      | 113,300.00             | 0.00         | 0.00         | 0.00         |
|                          | 30,000      | 113,300.00             | 0.00         | 0.00         | 0.00         |
|                          | 35,000      | 113,300.00             | 0.00         | 0.00         | 0.00         |
| Fertilizantes disponibles| 4,000       | 113,300.00             | 0.00         | 0.00         | 0.00         |
|                          | 5,000       | 113,300.00             | 0.00         | 0.00         | 0.00         |
|                          | 6,000       | 113,300.00             | 0.00         | 0.00         | 0.00         |

---

## Interpretación de los Resultados

1. **Superficie Cultivable**:
   - Cuando el valor asociado a la superficie cultivable es bajo ($50$ €/ha), el modelo identifica un precio sombra de $1500.00$ €/ha, indicando que la superficie es un recurso limitante en este caso.
   - Para valores superiores ($100$ €/ha o más), el precio sombra se reduce a $0.00$, lo que implica que este recurso deja de ser una limitación.

2. **Agua Disponible**:
   - El precio sombra del agua es $0.00$ en todos los casos analizados, lo que indica que la cantidad disponible es suficiente y no afecta el coste total mínimo.

3. **Fertilizantes Disponibles**:
   - Similar al agua, los fertilizantes no presentan un precio sombra significativo ($0.00$ en todos los casos). Esto sugiere que su disponibilidad actual no limita la solución óptima.

---

## Conclusión General

1. **Superficie como Recurso Crítico**:
   - La **superficie cultivable** es el único recurso que puede llegar a ser un limitante. Cuando su valor asociado disminuye, se refleja un precio sombra significativo, lo que sugiere que aumentarla sería beneficioso en condiciones más restrictivas.

2. **Abundancia de Agua y Fertilizantes**:
   - La disponibilidad de agua y fertilizantes es suficiente en este modelo, lo que se traduce en precios sombra de $0.00$ incluso al modificar sus valores asociados.

3. **Coste Total Mínimo**:
   - El coste total mínimo es **113,300.00 €** en la mayoría de los casos, salvo cuando la superficie cultivable tiene un coste muy bajo, lo que sugiere una dependencia leve del valor asociado a este recurso.

Este análisis reafirma la importancia de monitorear la disponibilidad de superficie cultivable como el recurso más crítico para garantizar una optimización robusta en este modelo.

# Grafo Primal

---

In [60]:
#ejecutar 
import plotly.graph_objects as go
import networkx as nx

# Crear el grafo para el problema primal
G_primal = nx.DiGraph()

# Agregar nodos (recursos y cultivos)
recursos = ["Superficie Total", "Agua", "Fertilizantes"]
cultivos = ["Tomates", "Pimientos", "Lechugas", "Calabacines", "Berenjenas", "Zanahorias"]

nodos_primal = recursos + cultivos
G_primal.add_nodes_from(nodos_primal)

# Agregar aristas (relaciones entre recursos y cultivos)
aristas_primal = [
    (r, c) for r in recursos for c in cultivos
]
G_primal.add_edges_from(aristas_primal)

# Resultados del problema primal
resultados_primal = {
    "Tomates": 31.00,
    "Pimientos": 15.00,
    "Lechugas": 26.50,
    "Calabacines": 8.00,
    "Berenjenas": 10.00,
    "Zanahorias": 7.00
}

# Posiciones manuales para nodos en capas
pos = {}
layer_distance = 1.5
for i, node in enumerate(recursos):
    pos[node] = (i * 2, 0, 0)  # Capa superior (recursos)
for i, node in enumerate(cultivos):
    pos[node] = (i * 2, -layer_distance, 0)  # Capa inferior (cultivos)

# Coordenadas para nodos y aristas
x_nodes = [pos[node][0] for node in nodos_primal]
y_nodes = [pos[node][1] for node in nodos_primal]
z_nodes = [pos[node][2] for node in nodos_primal]

x_edges = []
y_edges = []
z_edges = []

for edge in G_primal.edges():
    x_edges += [pos[edge[0]][0], pos[edge[1]][0], None]
    y_edges += [pos[edge[0]][1], pos[edge[1]][1], None]
    z_edges += [pos[edge[0]][2], pos[edge[1]][2], None]

# Crear etiquetas con los resultados del primal
labels = []
for node in G_primal.nodes():
    if node in resultados_primal:
        labels.append(f"{node}<br>{resultados_primal[node]} hectáreas")
    else:
        labels.append(node)

# Crear el grafo interactivo en 3D
fig = go.Figure()

# Agregar aristas
fig.add_trace(go.Scatter3d(
    x=x_edges, y=y_edges, z=z_edges,
    mode='lines',
    line=dict(color='gray', width=1),
    hoverinfo='none'
))

# Agregar nodos
fig.add_trace(go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers+text',
    marker=dict(size=10, color='blue'),
    text=labels,  # Etiquetas con los resultados del primal
    textposition="top center",
    hovertemplate='%{text}<extra></extra>'
))

# Configurar el diseño
fig.update_layout(
    title="Grafo del Problema Primal en 3D con Distribución Lógica",
    scene=dict(
        xaxis=dict(title="Eje X", showgrid=False, zeroline=False),
        yaxis=dict(title="Eje Y", showgrid=False, zeroline=False),
        zaxis=dict(title="Eje Z", showgrid=False, zeroline=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show(renderer='browser')    

# Grafo dual 
---

In [53]:
#ejecutar  
import plotly.graph_objects as go
import networkx as nx

# Crear el grafo para el problema dual
G_dual = nx.DiGraph()

# Nodos del dual
restricciones_dual = [
    "Tomates (2000)", "Pimientos (2500)", "Lechugas (1500)",
    "Calabacines (1800)", "Berenjenas (2200)", "Zanahorias (1700)"
]
variables_dual = [
    "Y1 (Superficie: 0 €/ha)", "Y2 (Agua: 0 €/m³)", "Y3 (Fertilizantes: 0 €/kg)",
    "Y4 (Tomates: 2000 €)", "Y5 (Pimientos: 2500 €)", "Y6 (Lechugas: 1500 €)",
    "Y7 (Calabacines: 1800 €)", "Y8 (Berenjenas: 2200 €)", "Y9 (Zanahorias: 1700 €)"
]

nodos_dual = restricciones_dual + variables_dual
G_dual.add_nodes_from(nodos_dual)

# Aristas del dual (relaciones entre restricciones y variables)
aristas_dual = [
    ("Tomates (2000)", "Y1 (Superficie: 0 €/ha)"),
    ("Tomates (2000)", "Y2 (Agua: 0 €/m³)"),
    ("Tomates (2000)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Tomates (2000)", "Y4 (Tomates: 2000 €)"),
    ("Pimientos (2500)", "Y1 (Superficie: 0 €/ha)"),
    ("Pimientos (2500)", "Y2 (Agua: 0 €/m³)"),
    ("Pimientos (2500)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Pimientos (2500)", "Y5 (Pimientos: 2500 €)"),
    ("Lechugas (1500)", "Y1 (Superficie: 0 €/ha)"),
    ("Lechugas (1500)", "Y2 (Agua: 0 €/m³)"),
    ("Lechugas (1500)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Lechugas (1500)", "Y6 (Lechugas: 1500 €)"),
    ("Calabacines (1800)", "Y1 (Superficie: 0 €/ha)"),
    ("Calabacines (1800)", "Y2 (Agua: 0 €/m³)"),
    ("Calabacines (1800)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Calabacines (1800)", "Y7 (Calabacines: 1800 €)"),
    ("Berenjenas (2200)", "Y1 (Superficie: 0 €/ha)"),
    ("Berenjenas (2200)", "Y2 (Agua: 0 €/m³)"),
    ("Berenjenas (2200)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Berenjenas (2200)", "Y8 (Berenjenas: 2200 €)"),
    ("Zanahorias (1700)", "Y1 (Superficie: 0 €/ha)"),
    ("Zanahorias (1700)", "Y2 (Agua: 0 €/m³)"),
    ("Zanahorias (1700)", "Y3 (Fertilizantes: 0 €/kg)"),
    ("Zanahorias (1700)", "Y9 (Zanahorias: 1700 €)")
]
G_dual.add_edges_from(aristas_dual)

# Posiciones manuales para nodos en capas
pos_dual = {}
layer_distance = 1.5
for i, node in enumerate(restricciones_dual):
    pos_dual[node] = (i * 2, 0, 0)  # Capa superior (restricciones)
for i, node in enumerate(variables_dual):
    pos_dual[node] = (i * 2, -layer_distance, 0)  # Capa inferior (variables)

# Coordenadas para nodos y aristas
x_nodes_dual = [pos_dual[node][0] for node in nodos_dual]
y_nodes_dual = [pos_dual[node][1] for node in nodos_dual]
z_nodes_dual = [pos_dual[node][2] for node in nodos_dual]

x_edges_dual = []
y_edges_dual = []
z_edges_dual = []

for edge in G_dual.edges():
    x_edges_dual += [pos_dual[edge[0]][0], pos_dual[edge[1]][0], None]
    y_edges_dual += [pos_dual[edge[0]][1], pos_dual[edge[1]][1], None]
    z_edges_dual += [pos_dual[edge[0]][2], pos_dual[edge[1]][2], None]

# Crear etiquetas para los nodos del dual
labels_dual = nodos_dual  # Los nombres de los nodos ya contienen la información

# Crear el grafo interactivo en 3D
fig_dual = go.Figure()

# Agregar aristas
fig_dual.add_trace(go.Scatter3d(
    x=x_edges_dual, y=y_edges_dual, z=z_edges_dual,
    mode='lines',
    line=dict(color='gray', width=1),
    hoverinfo='none'
))

# Agregar nodos
fig_dual.add_trace(go.Scatter3d(
    x=x_nodes_dual, y=y_nodes_dual, z=z_nodes_dual,
    mode='markers+text',
    marker=dict(size=10, color='orange'),
    text=labels_dual,  # Etiquetas con los precios sombra y restricciones
    textposition="top center",
    hovertemplate='%{text}<extra></extra>'
))

# Configurar el diseño
fig_dual.update_layout(
    title="Grafo del Problema Dual en 3D con Precios Sombra",
    scene=dict(
        xaxis=dict(title="Eje X", showgrid=False, zeroline=False),
        yaxis=dict(title="Eje Y", showgrid=False, zeroline=False),
        zaxis=dict(title="Eje Z", showgrid=False, zeroline=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig_dual.show(renderer='browser')

# Problema: Optimización Binaria en la Distribución de Productos

## Contexto

Una granja produce tres cultivos principales: **Tomates**, **Pimientos** y **Lechugas**. Cada cliente tiene demandas específicas, pero la distribución debe decidirse entre opciones binarias de transporte para optimizar el beneficio total. Cada decisión implica enviar o no productos a cada cliente, maximizando así la utilidad total de la distribución.

---

## Planteamiento del Problema

### Variables de Decisión

- $x_{ij}$: Variable binaria que toma el valor $1$ si se envía el producto $j$ al cliente $i$, y $0$ en caso contrario.
  - $i \in \{\text{Cliente 1, Cliente 2, Cliente 3}\}$
  - $j \in \{\text{Tomates, Pimientos, Lechugas}\}$

---

### Función Objetivo

Maximizar el beneficio total:
$$
Z = 8x_{\text{Cliente 1, Tomates}} + 5x_{\text{Cliente 1, Pimientos}} + 6x_{\text{Cliente 1, Lechugas}} +
4x_{\text{Cliente 2, Tomates}} + 7x_{\text{Cliente 2, Pimientos}} + 3x_{\text{Cliente 2, Lechugas}} +
9x_{\text{Cliente 3, Tomates}} + 2x_{\text{Cliente 3, Pimientos}} + 4x_{\text{Cliente 3, Lechugas}}
$$

---

### Restricciones

1. **Capacidad Total**:
   - El envío total por producto no puede superar la capacidad de producción:
     $$
     x_{\text{Cliente 1, Tomates}} + x_{\text{Cliente 2, Tomates}} + x_{\text{Cliente 3, Tomates}} \leq 1
     $$
     $$
     x_{\text{Cliente 1, Pimientos}} + x_{\text{Cliente 2, Pimientos}} + x_{\text{Cliente 3, Pimientos}} \leq 1
     $$
     $$
     x_{\text{Cliente 1, Lechugas}} + x_{\text{Cliente 2, Lechugas}} + x_{\text{Cliente 3, Lechugas}} \leq 1
     $$

2. **Demanda de Clientes**:
   - Cada cliente puede recibir un máximo de un envío por producto:
     $$
     x_{\text{Cliente 1, Tomates}} + x_{\text{Cliente 1, Pimientos}} + x_{\text{Cliente 1, Lechugas}} \leq 1
     $$
     $$
     x_{\text{Cliente 2, Tomates}} + x_{\text{Cliente 2, Pimientos}} + x_{\text{Cliente 2, Lechugas}} \leq 1
     $$
     $$
     x_{\text{Cliente 3, Tomates}} + x_{\text{Cliente 3, Pimientos}} + x_{\text{Cliente 3, Lechugas}} \leq 1
     $$

3. **Variables Binarias**:
   - Las variables $x_{ij}$ son binarias:
     $$
     x_{ij} \in \{0, 1\} \quad \text{para todo } i, j.
     $$

---

## Objetivo

Determinar las decisiones binarias óptimas para maximizar el beneficio total, cumpliendo las restricciones de capacidad y demanda.

In [54]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjuntos
productos = ["Tomates", "Pimientos", "Lechugas"]
clientes = ["Cliente 1", "Cliente 2", "Cliente 3"]

# Variables de decisión binarias
model.x = Var(clientes, productos, domain=Binary)

# Parámetros de beneficios
beneficios = {
    ("Cliente 1", "Tomates"): 8, ("Cliente 1", "Pimientos"): 5, ("Cliente 1", "Lechugas"): 6,
    ("Cliente 2", "Tomates"): 4, ("Cliente 2", "Pimientos"): 7, ("Cliente 2", "Lechugas"): 3,
    ("Cliente 3", "Tomates"): 9, ("Cliente 3", "Pimientos"): 2, ("Cliente 3", "Lechugas"): 4
}

# Función objetivo: Maximizar el beneficio total
model.obj = Objective(
    expr=sum(model.x[i, j] * beneficios[i, j] for i in clientes for j in productos),
    sense=maximize
)

# Restricciones de capacidad por producto
model.capacidad = ConstraintList()
for j in productos:
    model.capacidad.add(sum(model.x[i, j] for i in clientes) <= 1)

# Restricciones de demanda por cliente
model.demanda = ConstraintList()
for i in clientes:
    model.demanda.add(sum(model.x[i, j] for j in productos) <= 1)

# Resolver el modelo
solver = SolverFactory("glpk")
result = solver.solve(model, tee=False)

# Mostrar resultados
print("\n🔹 Resultados 🔹")
if result.solver.termination_condition == 'optimal':
    print(f"\nBeneficio total máximo: {model.obj():.2f} €")
    print("\nDecisiones óptimas:")
    for i in clientes:
        for j in productos:
            if model.x[i, j].value > 0:
                print(f"  - Enviar {j} a {i}")
else:
    print("\n❌ No se encontró solución óptima.")


🔹 Resultados 🔹

Beneficio total máximo: 22.00 €

Decisiones óptimas:
  - Enviar Lechugas a Cliente 1
  - Enviar Pimientos a Cliente 2
  - Enviar Tomates a Cliente 3


# Conclusiones del Modelo de Distribución

## Análisis de los Resultados

El modelo ha permitido identificar una distribución óptima de productos agrícolas que maximiza el beneficio total, cumpliendo con las restricciones de capacidad y demanda.

### Beneficio Total
El **beneficio total máximo obtenido** es de **22.00 €**, lo que representa la solución óptima dentro de las condiciones establecidas.

### Decisiones Óptimas
Las decisiones de envío son las siguientes:
- **Lechugas** enviadas a **Cliente 1**.
- **Pimientos** enviados a **Cliente 2**.
- **Tomates** enviados a **Cliente 3**.

### Evaluación de las Restricciones
1. **Capacidades de Producción**: Se respetaron las capacidades máximas para cada producto.
2. **Demanda de Clientes**: Cada cliente recibió un producto, cumpliendo las condiciones del problema.

---

## Conclusión General

El modelo es eficiente para determinar decisiones binarias en problemas de distribución, proporcionando una solución clara y optimizada. Esto asegura un beneficio máximo dentro de un sistema logístico sencillo y escalable.

# Fuentes y Referencias


1. Campos, J. (s.f.). *Linear Programming*. Universidad de Zaragoza. Recuperado de [https://webdiis.unizar.es/~jcampos/ab/material/7-LinearProgramming.pdf](https://webdiis.unizar.es/~jcampos/ab/material/7-LinearProgramming.pdf)

2. Departamento de Matemática Aplicada, Universidad de Barcelona. (s.f.). *Optimización económica: Programación Lineal*. Recuperado de [http://www.ub.edu/matheopt/optimizacion-economica/programacion-lineal](http://www.ub.edu/matheopt/optimizacion-economica/programacion-lineal)

3. Python Software Foundation. (2023). *Pyomo Optimization Library*. Recuperado de [https://pyomo.org/](https://pyomo.org/)

4. Plotly Technologies Inc. (2023). *Plotly Python Graphing Library*. Recuperado de [https://plotly.com/python/](https://plotly.com/python/)

5. Hagberg, A., Schult, D., & Swart, P. (2008). *NetworkX: High-productivity software for complex networks*. Recuperado de [https://networkx.org/](https://networkx.org/)

6. OpenAI. (2025).  

---
